In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
import warnings
warnings.filterwarnings("ignore") 
from tqdm import tqdm_notebook
import os, re, sys, gc, pickle, time
from collections import defaultdict
from sklearn.metrics import make_scorer, roc_auc_score
import pandas as pd
import numpy as np
from itertools import combinations
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
from plot_metric.functions import BinaryClassification
import warnings
warnings.filterwarnings("ignore") 
from tqdm import tqdm_notebook
import os, re, sys, gc, pickle, time
from collections import defaultdict
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import stats
from sklearn.model_selection import GridSearchCV, cross_val_score,\
                                    KFold, train_test_split, cross_validate, ParameterGrid,\
                                    cross_validate, cross_val_predict, TimeSeriesSplit, StratifiedKFold
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score
import statsmodels.api as sm
from sklearn.base import BaseEstimator, TransformerMixin,  clone
from sklearn.metrics import confusion_matrix
# пайплайн
from sklearn.pipeline import Pipeline, make_union, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin,  clone
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, OneHotEncoder
from sklearn.feature_selection import SelectFromModel
# дамми-регрессор
from sklearn.dummy import DummyClassifier
from itertools import combinations
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import time
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_curve
import shutil
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [17]:
class SklearnHelperLogitSelector(BaseEstimator, TransformerMixin):
    ''' Отбор признаков '''
    def __init__(self, model, cv, scoring, show_progress):
        self.model = model
        self.cv = cv
        self.scoring = scoring
        self.show_progress = show_progress
    def fit(self, X, y=None):
        #####################################################################################################
        def calc_vif(X): 
            _X = pd.DataFrame(X)
            vif = pd.DataFrame()
            vif["variables"] = _X.columns
            vif["VIF"] = [variance_inflation_factor(_X.values, i) for i in tqdm_notebook(range(_X.shape[1]))]
            return(vif)
        ######################################################################################################
        _X = X.copy()
        cv_scores = []
        for i in tqdm_notebook(range(_X.shape[1])):
            try:
                _X_curr = _X[:, i].toarray().reshape(-1,1)
            except:
                _X_curr = _X[:, i].reshape(-1,1)                
            mean_cv_score = cross_val_score(self.model, _X_curr, y, cv =self.cv, scoring = self.scoring, n_jobs=-1).mean()            
            cv_scores.append(mean_cv_score)
            
        order = np.argsort(cv_scores)[::-1]
        to_drop_before, best_features, best_cv_score = [], [order[0]], -np.inf
        for i in tqdm_notebook(order[1:]):
            curr_features = best_features+[i]
            _X_curr = _X[:, curr_features]
            mean_cv_score = cross_val_score(self.model, _X_curr, y, cv =self.cv, scoring = self.scoring, n_jobs=-1).mean()
            if mean_cv_score>best_cv_score:
                sm_logit = sm.Logit(y, _X_curr).fit()
                pVals = np.array(sm_logit.pvalues)
                if np.all(pVals<=.05):                    
                    best_cv_score = mean_cv_score
                    best_features = curr_features
                    if self.show_progress:
                        print('new best score = {:.10f}'.format(best_cv_score))
            else:
                to_drop_before.append(i)
        while True:
            to_drop_after = []
            for i in tqdm_notebook(to_drop_before):
                curr_features = best_features+[i]
                _X_curr = _X[:, curr_features]
                mean_cv_score = cross_val_score(self.model, _X_curr, y, cv =self.cv, scoring = self.scoring, n_jobs=-1).mean()
                if mean_cv_score>best_cv_score:
                    sm_logit = sm.Logit(y, _X_curr).fit()
                    pVals = np.array(sm_logit.pvalues)
                    if np.all(pVals<=.05):
                        vif = calc_vif(_X_curr)
                        if (vif['VIF']<=10).all():
                            best_cv_score = mean_cv_score
                            best_features = curr_features
                            if self.show_progress:
                                print('new best score = {:.10f}'.format(best_cv_score))
                else:
                    to_drop_after.append(i)
            if to_drop_before == to_drop_after:
                break
            else:
                to_drop_before = to_drop_after  
        self.best_features_ = best_features
        self.best_score_ = best_cv_score
    def transform(self, _X):
        return _X[:, self.best_features_]

In [ ]:
X_tr = pd.read_pickle('X_tr.pickle')
X_te = pd.read_pickle('X_te.pickle')
y_tr = pd.read_pickle('y_tr.pickle')
y_te = pd.read_pickle('y_te.pickle')

In [15]:
SEED = 13
logit = LogisticRegression(random_state= SEED)
SKF = StratifiedKFold(5, shuffle = True, random_state = SEED)

In [26]:
ppl = make_pipeline(MinMaxScaler(),\
                    SimpleImputer(strategy='constant', fill_value=-1),\
                    SklearnHelperLogitSelector(model=logit, cv=SKF, scoring='roc_auc', show_progress=True))
ppl.fit(X_tr, y_tr)

In [18]:
_x_tr = ppl.transform(X_tr)
_x_te = ppl.transform(X_te)
_y_tr = y_tr.values
_y_te = y_te.values

In [27]:
logit_gs = GridSearchCV(LogisticRegression(),\
                         param_grid = {'C':np.logspace(-3, 3, 500),
                                        'random_state': [SEED],\
                                        'solver' : ['liblinear'],\
                                        'penalty':['l1','l2']},\
                        scoring ='roc_auc', cv = SKF, verbose = 1,n_jobs=-1)
logit_gs.fit(_x_tr, _y_tr)
GINI_best_cv = 100*((2 *logit_gs.best_score_)-1)

In [23]:
GINI_best_cv

50.50784319223065

In [25]:
y_predprob_te = pd.DataFrame(logit_gs.best_estimator_.predict_proba(_x_te)[:,1], columns= ['predprob'])
y_predprob_te['ucdb_id'] = y_te.index
y_predprob_te = y_predprob_te[['ucdb_id', 'predprob']]
y_predprob_te.to_excel('y3_predprob_te_v4.xlsx')